## First Application Of Tapas Model

In [ ]:
import pandas as pd

data = pd.read_excel("DataExtraction/MS_IS_all_modules.xlsx")
data.head()


from transformers import TapasForQuestionAnswering, TapasTokenizer
from transformers import pipeline

tqa = pipeline(task='table-question-answering', model='google/tapas-base-finetuned-wtq')


table = pd.read_excel("DataExtraction/MS_IS_all_modules.xlsx")
table = table.astype(str)

query = ["What's the module title of the module with abbreviation 12-IV-161-m01?"]
query = ["What ist the workload of Module title IT-Management?"]
answer = tqa(table=table, query=query)
print(answer['answer'].strip())

# FT Versuch

In [101]:
import pandas as pd
from transformers import TapasForQuestionAnswering, TapasTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
import torch

# Load the pre-trained model and tokenizer
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base-finetuned-wtq')
tokenizer = TapasTokenizer.from_pretrained('google/tapas-base-finetuned-wtq')

# Prepare your own training data in the format expected by the model
table = [
    ['Name', 'Age', 'Gender'],
    ['John', '25', 'Male'],
    ['Jane', '30', 'Female']
]


df = pd.DataFrame(table[1:], columns=table[0])

queries = ["What's the Age of John?"]
answer_text = ["25"]
answer_coordinates = [(1, 1, 1, 1)] 


# Tokenize the training data using the TapasTokenizer
inputs = tokenizer(
    table=df,
    queries=queries,
    answer_text=answer_text,
    answer_coordinates=answer_coordinates,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,           
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64, 
    warmup_steps=500,                
    weight_decay=0.01,            
    logging_dir='./logs',           
    logging_steps=10,
)

# Define the trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=inputs,               
    data_collator=lambda data: {'input_ids': torch.stack([x['input_ids'] for x in data]),
                                'attention_mask': torch.stack([x['attention_mask'] for x in data]),
                                'token_type_ids': torch.stack([x['token_type_ids'] for x in data]),
                                'labels': torch.stack([x['labels'] for x in data])},
    tokenizer=tokenizer,                 # tokenizer to be used for tokenization
)

trainer.train()

trainer.save_model('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-tokenizer')


TypeError: 'int' object is not subscriptable

In [ ]:
import mlflow

# Start an MLflow run
mlflow.start_run()

# Log your hyperparameters
mlflow.log_params({
    "learning_rate": 1e-5,
    "batch_size": 32,
    "num_epochs": 1
})

# Step 4: Model Training
# ...

# Set up your training loop
for epoch in range(num_epochs):
    model.train()
    # ...

    for batch in DataLoader(dataset, batch_size=batch_size, shuffle=True):
        # ...

        # Compute the loss
        loss = outputs.loss

        # Log the loss metric
        mlflow.log_metric("loss", loss.item(), step=epoch)

        # ...

# Step 5: Fine-tuning Parameters
# ...

# Log your trained model as an artifact
mlflow.pytorch.log_model(model, "tapas_model")

# End the MLflow run
mlflow.end_run()
